#                         02807 Computational Tools for Data Science
##                                                      Final report 

group 26
Yanan Zhao & s210319
Max Specktor & s184362
Malthe Dohm Andersen & s194257
Kevin Thieu & s221885

**************************************************************************************************************

# Install packages

In [ ]:
# basic package
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# for visual 
import folium
import dataframe_image as dfi
# for clustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score as ss
import itertools
from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt
import random
from shapely.geometry import MultiPoint
from geopy.distance import great_circle
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import calinski_harabasz_score
import mmh3
from nltk.corpus import stopwords
import math
from tqdm import tqdm

#For faster computation
from numba import jit 

#for deep learning:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torch.nn.parameter import Parameter
from torch.autograd import Variable
from sklearn.metrics import accuracy_score, confusion_matrix
import pickle
from matplotlib.pylab import (figure, semilogx, loglog, xlabel, ylabel, legend, 
                           title, subplot, show, grid)
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import sklearn.linear_model as lm

---------------------------------------------------------------------------

# Data processing 
Visualise data, pick interesting attributes, and transformed.

In [ ]:
#load data
df = pd.read_csv('../../US_Accidents_May19_Migrated Data.csv')

In [ ]:
#dataset observing
df.shape
df.head()
df.isnull().sum()

In [ ]:
# selecting attribute
df.drop = df[['ID','City','State','Severity','Visibility(mi)','Start_Lat','Start_Lng', 
            'count Traffic Signal','Count of Crossing','count of Bump','Description','Count of accidents',
             'Weather_Condition','Humidity(%)','Precipitation(in)','Wind_Chill(F)','Wind_Speed(mph)']]
df.drop = pd.get_dummies(df.drop, columns=['Amenity', 
    'Bump', 
    'Crossing',
    'Give_Way', 
    'Junction', 
    'No_Exit',
    'Railway', 
    'Roundabout', 
    'Station',
    'Stop', 
    'Traffic_Calming',
    'Traffic_Signal', 
    'Turning_Loop'])

In [ ]:
# print description table as a plot
table = df.drop.describe()

dfi.export(table, 'dataframe.png')
table

In [ ]:
# visilize count of accident in each state
states = df.State.unique()
count_by_state=[]
for i in df.State.unique():
    count_by_state.append(df[df['State']==i].count()['ID'])

fig,ax = plt.subplots(figsize=(16,10))
sns.barplot(states,count_by_state)

In [ ]:
#find top 10 city having accident
top_cities=df["City"].value_counts().sort_values()[-20:].reset_index()
top_cities.columns=["city","number_of_accidents"]

plt.figure(figsize=(10,7))
sns.barplot(x="city",y="accidents number",data=top_cities)
plt.title("TOP 10 CITIES WITH HIGHEST NUMBER OF ACCIDENTS",fontsize=20)
plt.xticks(rotation=40)
plt.show()

In [ ]:
# map represent accident severity 

severity_cols = {
    0: 'green',
    1: 'palegreen',
    2: 'papayawhip',
    3: 'lightsalmon',
    4: 'tomato'
}

vcol = [severity_cols[i] for i in df['Severity']]

ax = plt.scatter(df['Start_Lng'], df['Start_Lat'],c = vcol,s=2)
plt.title('Accidents representating map by severity level')
fig = ax.get_figure()
fig.savefig('Severity.png')

In [ ]:
# percentage of accident including road params, save as plot
road_params = [
    'Amenity', 
    'Bump', 
    'Crossing',
    'Give_Way', 
    'Junction', 
    'No_Exit',
    'Railway', 
    'Roundabout', 
    'Station',
    'Stop', 
    'Traffic_Calming',
    'Traffic_Signal', 
    'Turning_Loop']

# % of accident including road params
road_param_percent = df.loc[:, road_params].sum() / len(df)
plt.title('Presence of road element near accidents')
plt.xlabel('% of total of accidents')
ax=road_param_percent.sort_values().plot(kind='barh');

fig = ax.get_figure()
fig.savefig('road.png')

In [ ]:
# percentage of accident by Weather_Condition
acc_by_weather_condition = df.groupby('Weather_Condition').size() / len(df)
acc_by_weather_condition = acc_by_weather_condition[acc_by_weather_condition > 0.005]
plt.title('Presence of weather condition during accidents')
plt.xlabel('% of total of accidents')
acc_by_weather_condition.sort_values().plot(kind='barh');

---------------------------------------------------------------------------

# Clustering

## K-means

### New york

In [ ]:
#Get data with pd
"""Take in the file"""
df = pd.read_csv('US_Accidents_May19_Migrated Data.csv')
df.head()

In [ ]:
city = df.loc[df['City'] == "New York"]
ny_lat = city["Start_Lat"]
ny_lng = city["Start_Lng"]

In [ ]:
def comb(lat : list ,long : list) -> list:
    data = np.column_stack((lat,long))
    return data

NY_coord = comb(ny_lat,ny_lng)
print(NY_coord[:,1])

In [ ]:
plt.scatter(NY_coord[:,0],NY_coord[:,1],s=2)

In [ ]:
#Elbow plot
def optimise_k_means(data, max_k): 
  means = []
  inertias = []
  
  for k in range(1,max_k):
      kmeans = KMeans(n_clusters = k) 
      kmeans.fit(data) 
      
      means.append(k)
      inertias.append(kmeans.inertia_)#Look further in to this
      
  #Generate plot
  fig = plt.subplots(figsize = (10,5))
  plt.plot(means,inertias, "o-")
  plt.xlabel("Numbers of Clusters")
  plt.ylabel("Inertia")
  plt.grid(True)
  plt.show()
  
optimise_k_means(NY_coord,10)

In [ ]:
#Tried to make a subplot, but does not work
def comparing_n_cluster(max_cluster : int, data):

    list_of_means = []
    x_pt = np.array([row[0] for row in data])
    y_pt = np.array([row[1] for row in data])
    for k in range(1,max_cluster):
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(data)
        kmeans_label = kmeans.labels_
        list_of_means.append(kmeans_label)
    
    for i in range(1,max_cluster-1):
        plt.scatter(x = x_pt, y = y_pt , c = list_of_means[i])
        plt.show()
    
        
comparing_n_cluster(6, NY_coord)

In [ ]:
DBS_score = []
for_plot = []
for i in range(2,10):
    kmeans = KMeans(n_clusters=i, random_state=1).fit(NY_coord)
    labels = kmeans.labels_
    score = davies_bouldin_score(NY_coord,labels)
    DBS_score.append(score)
    for_plot.append(i)

plt.plot(for_plot,DBS_score,"o-")
plt.show()

print(min(DBS_score))
print(np.argmin(DBS_score)+2)


In [ ]:
kmeans = KMeans(n_clusters=3).fit(NY_coord)
labels = kmeans.labels_
ch_index = calinski_harabasz_score(NY_coord,labels)

In [ ]:
print(ch_index)

In [ ]:
sil_score = ss(NY_coord,labels)

In [ ]:
print(sil_score)

In [ ]:
ny_lat = NY_coord[:,0]
ny_lng = NY_coord[:,1]
location = ny_lat.mean(), ny_lng.mean()
m = folium.Map(location=location,zoom_start=11.5,control_scale = True)
folium.TileLayer('cartodbpositron').add_to(m)
#plugins.MarkerCluster(NY_involved[['latitude','longitude']]).add_to(m)
#folium.plugins.HeatMap(NY_involved).add_to(m)

kmeans = KMeans(n_clusters = 3)
kmeans.fit(NY_coord)
kmeans_label = kmeans.labels_


clust_colours = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99','#b15928']

for i in range(0,len(NY_coord)):
    colouridx = kmeans_label[i]
    if colouridx == -1:
        folium.CircleMarker([ny_lat[i],ny_lng[i]], radius = 5, color = "white", fill = "white").add_to(m)
    else:
        col = clust_colours[colouridx%len(clust_colours)]
        folium.CircleMarker([ny_lat[i],ny_lng[i]], radius = 5, color = col, fill = col).add_to(m)
        

m

## US

In [ ]:
#Get data with pd
"""Take in the file"""
df = pd.read_csv('US_Accidents_May19_Migrated Data.csv')
df.head()

In [ ]:
#Acess wanted column and convert to numpy
latitude = np.array(df["Start_Lat"])
longitude = np.array(df["Start_Lng"])

In [ ]:
#To see if latitude and longitude match in case some data are lost or NaN
@jit(nopython = True)
def match(lat,lng) -> bool:
    if (len(latitude) == len(longitude)):
        return True
    else:
        return False
    
print(match(latitude,longitude))

In [ ]:
#To get easy acess of the data
np.savetxt(r'lng.txt', longitude, fmt='%1.6f')
np.savetxt(r'lat.txt', latitude, fmt='%1.6f')

In [ ]:
#Combining the latitude og longitude together
@jit 
def comb(lat : list ,long : list) -> list:
    data = np.column_stack((lat,long))
    return data

data = comb(latitude,longitude)

In [ ]:
#Just in case, the coordinates are saved to a file
np.savetxt(r'Points.txt', data, fmt='%1.6f') 

In [ ]:
#Choosing random points with seed(10)
def random_sample() -> list:
    l = list(range(2243939))
    random.seed(10)
    sorted_list = sorted(random.sample(l, 100000))
    return sorted_list

pick = np.array(random_sample())

In [ ]:
#Create an subset from the random sample
#@jit
def data_sub_set(random_list : list , data_set : list) -> list:
    sub_set = np.empty([len(random_list),2])
    iterator = 0
    for i in random_list:
        sub_set[iterator] = data_set[i]
        iterator += 1
    return sub_set

new_set = data_sub_set(pick,data)

In [ ]:
#To see how the random data is spread out on a grid
new_set = data_sub_set(pick,data)
new_lat = np.array([row[0] for row in new_set])
new_lng = np.array([row[1] for row in new_set])


plt.scatter(new_lat ,new_lng,s=4)
plt.grid(True)
plt.show()

In [ ]:
#To Calculate number of optimal k means
#This function shows how many cluster point we should have. 
def optimise_k_means(data, max_k): 
  means = []
  inertias = []
  
  for k in range(1,max_k):
      kmeans = KMeans(n_clusters = k) 
      kmeans.fit(data) 
      
      means.append(k)
      inertias.append(kmeans.inertia_)
      
  #Generate plot
  fig = plt.subplots(figsize = (10,5))
  plt.plot(means,inertias, "o-")
  plt.xlabel("Numbers of Clusters")
  plt.ylabel("Inertia")
  plt.grid(True)
  plt.show()
  
optimise_k_means(new_set,10)

In [ ]:
#Calculate silhoutte_method

def silhoutte_method(data, max_k):
    sil = []
    for k in range(2,max_k+1):
        kmeans = KMean(n_clusters = k).fit(data)
        labels = kmeans.labels_
        score = ss(data,labels)
        sil.append(score)
    return sil

In [ ]:
sil_score = silhoutte_method(new_set, 3)

In [ ]:
print(sil_score)

In [ ]:
#Calcualte Calinski_harabasz method
kmeans = KMeans(n_clusters=3, random_state=1).fit(new_set)
labels = kmeans.labels_
ch_index = calinski_harabasz_score(new_set,labels )

In [ ]:
print(ch_index)

In [ ]:
DBS_score = []
for_plot = []
for i in range(2,15):
    kmeans = KMeans(n_clusters=i, random_state=1).fit(new_set)
    labels = kmeans.labels_
    score = davies_bouldin_score(new_set, labels)
    DBS_score.append(score)
    for_plot.append(i)

plt.plot(for_plot,DBS_score,"o-")
plt.show()

In [ ]:
#Get score and value
print(min(DBS_score))
print(np.argmin(DBS_score)+2)

In [ ]:
#Visualise and compare different size of cluster
def comparing_n_cluster(max_cluster : int, data):

    list_of_means = []
    x_pt = np.array([row[0] for row in new_set])
    y_pt = np.array([row[1] for row in data])
    for k in range(1,max_cluster):
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(data)
        kmeans_label = kmeans.labels_
        list_of_means.append(kmeans_label)
    
    #fig, axs = plt.subplots(nrows=1, ncols=5, figsize=(20,5))
    
    #ist_of_means = list(list_of_means)
    
    #for i, ax in enumerate(fig.axes,start=1):
    #    ax.scatter(x = np.array(row[0] for row in data), y = np.array(row[1] for row in data))
    for i in range(1,max_cluster-1):
        plt.scatter(x = x_pt, y = y_pt , c = list_of_means[i])
        plt.show()
    
        
comparing_n_cluster(6, new_set)

In [ ]:
#plotting on a map
location = new_lat.mean(), new_lng.mean()

m = folium.Map(location=location,zoom_start=11.5,control_scale = True)
folium.TileLayer('cartodbpositron').add_to(m)

kmeans = KMeans(n_clusters = 3)
kmeans.fit(new_set)
kmeans_label = kmeans.labels_

clust_colours = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99','#b15928']

for i in range(0,len(new_set)):
    colouridx = kmeans_label[i]
    if colouridx == -1:
        folium.CircleMarker([new_lat[i],new_lng[i]], radius = 5, color = "white", fill = "white").add_to(m)
    else:
        col = clust_colours[colouridx%len(clust_colours)]
        folium.CircleMarker([new_lat[i],new_lng[i]], radius = 5, color = col, fill = col).add_to(m)
        

m

## DBSCAN
### New york
pick coordinates in NY, map records, use k-distance plot knee point to find optimal eps range

In [ ]:
# take lat and lng colunm filter Newyork data
NY = df['City'] == 'New York'
NY_df = df[NY]
NY_loca_df = NY_df[['Start_Lat','Start_Lng']]
NY_loca_df.columns = ["latitude", "longitude"]
coords = NY_loca_df[["latitude", "longitude"]]
X = NY_loca_df.to_numpy()

In [ ]:
#simply plot new york accident coordinate 
plt.scatter( NY_loca_df["longitude"],NY_loca_df["latitude"],s=2)

In [ ]:
# NearestNeighbors find knee point for optimal eps
neigh = NearestNeighbors(n_neighbors=5)
nbrs = neigh.fit(np.radians(X))
distances, indices = nbrs.kneighbors(np.radians(X))
distances = distances[:, 1]
distances = np.sort(distances, axis=0)
fig=plt.figure()
plt.plot(distances)
plt.xlim(4000, 5570)

In [ ]:
# first time try 
dbscan_cluster_model = DBSCAN(eps=0.000035, min_samples=5, algorithm='ball_tree', metric='haversine').fit(np.radians(X))
dbscan_cluster_model
dbscan_cluster_model.labels_
NY_loca_df['cluster'] = dbscan_cluster_model.labels_
location = NY_loca_df['latitude'].mean(), NY_loca_df['longitude'].mean()

m = folium.Map(location=location,zoom_start=11,control_scale = True)

folium.TileLayer('cartodbpositron').add_to(m)

clust_colours = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99','#b15928']

for i in range(0,len(NY_loca_df)):
    colouridx = NY_loca_df['cluster'].iloc[i]
    if colouridx == -1:
        pass
    else:
        col = clust_colours[colouridx%len(clust_colours)]
        folium.CircleMarker([NY_loca_df['latitude'].iloc[i],NY_loca_df['longitude'].iloc[i]], radius = 10, color = col, fill = col).add_to(m)

m

With range pf epsilon, We use it for the parameter going forward and try to find the optimal value of MinPts based on Silhouette score. There are several combinations of eps and min_samples to see if we can get best Silhouette score and reasonable clusters. the cose is inspired by https://colab.research.google.com/drive/1DphvjpgQXwBWQq08dMyoSc6UREzXLxSE?usp=sharing#scrollTo=LyXo0mgdOTc1

In [ ]:
# find optimal min_samples based on 

ss(X, NY_loca_df['cluster'])
epsilons = np.linspace(6.75e-05,8e-05, num=3)
print(epsilons)
min_samples = np.arange(2, 100 , step=5) 
print(min_samples)
combinations = list(itertools.product(epsilons, min_samples))
print(combinations)
N = len(combinations)

#define a function to run through all combinations
def get_scores_and_labels(combinations, X):
  scores = []
  all_labels_list = []

  for i, (eps, num_samples) in enumerate(combinations):
    
    dbscan_cluster_model = DBSCAN(eps= eps, min_samples= num_samples, algorithm='ball_tree', metric='haversine').fit(np.radians(X))
    labels = dbscan_cluster_model.labels_
    labels_set = set(labels)
    num_clusters = len(labels_set)
    if -1 in labels_set:
      num_clusters -= 1
    
    if (num_clusters < 2) or (num_clusters > 50):
      scores.append(-10)
      all_labels_list.append('bad')
      c = (eps, num_samples)
      print(f"Combination {c} on iteration {i+1} of {N} has {num_clusters} clusters. Moving on")
      continue
    
    scores.append(ss(X, labels))
    all_labels_list.append(labels)
    print(f"Index: {i}, Score: {scores[-1]}, Labels: {all_labels_list[-1]}, NumClusters: {num_clusters}")

  best_index = np.argmax(scores)
  best_parameters = combinations[best_index]
  best_labels = all_labels_list[best_index]
  best_score = scores[best_index]

  return {'best_epsilon': best_parameters[0],
          'best_min_samples': best_parameters[1], 
          'best_labels': best_labels,
          'best_score': best_score}

# find best model
best_dict = get_scores_and_labels(combinations, X)
NY_loca_df['cluster'] = best_dict['best_labels']
best_dict

In [ ]:
# pick clustered data excluding outliers
involved = NY_loca_df['cluster'] != -1
NY_involved = NY_loca_df[involved]
NY_involved

Xx = NY_involved[['latitude','longitude']].to_numpy()
lablel = NY_involved[['cluster']]

In [ ]:
# get clusters centoid 
num_clusters = len(set(best_dict['best_labels']) - set([-1]))
cluster_labels = best_dict['best_labels']
clusters = pd.Series([X[cluster_labels == n] for n in range(num_clusters)])
clusters

#clusters centoid 
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

# get the centroid point for each cluster
centermost_points = clusters.map(get_centermost_point)
lats, lons = zip(*centermost_points)
rep_points = pd.DataFrame({'lon':lons, 'lat':lats})
rep_points

use folium map all records including noise as white circle, also mark cluster centriods

In [ ]:
# Visualise clusters in map
location = NY_loca_df['latitude'].mean(), NY_loca_df['longitude'].mean()

m = folium.Map(location=location,zoom_start=11,control_scale = True)
folium.TileLayer('cartodbpositron').add_to(m)


for i in range(0,len(NY_loca_df)):
    colouridx = NY_loca_df['cluster'].iloc[i]
    if colouridx == -1:
        folium.CircleMarker([NY_loca_df['latitude'].iloc[i],NY_loca_df['longitude'].iloc[i]], radius = 5, color = "white", fill = "white").add_to(m)
    else:
        col = clust_colours[colouridx%len(clust_colours)]
        folium.CircleMarker([NY_loca_df['latitude'].iloc[i],NY_loca_df['longitude'].iloc[i]], radius = 5, color = col, fill = col).add_to(m)
        
for i in range(len(rep_points)):
    folium.CircleMarker([rep_points['lat'].iloc[i],rep_points['lon'].iloc[i]], radius = 2, color = "black", fill_opacity=0.7, fill = "black").add_to(m)      
        
m.save("ny.html")

caculate DBscore and CI score

In [ ]:
# davies_bouldin_score
db_index = davies_bouldin_score(X, best_dict['best_labels'])
db_index

In [ ]:
#ch_index
ch_index = calinski_harabasz_score(X, best_dict['best_labels'])
print(ch_index)

### US
the codes are very same as what we did on NY data

In [ ]:
# take lat and lng colunm 
US_loca_df = df[['Start_Lat','Start_Lng']]
US_loca_df.columns = ["latitude", "longitude"]
coords = US_loca_df[["latitude", "longitude"]]
X = US_loca_df.to_numpy()

# random select 100000 samples from entire US data
l = list(range(2243939))
random.seed(10)
pick = sorted(random.sample(l, 100000))

In [ ]:
##simply plot US accident coordinate 
plt.scatter( US_picked_df["longitude"],US_picked_df["latitude"],s=2)

In [ ]:
#elbow method define range of eps
neigh = NearestNeighbors(n_neighbors=300)
nbrs = neigh.fit(np.radians(X))
distances, indices = nbrs.kneighbors(np.radians(X))
distances = distances[:, 1]
distances = np.sort(distances, axis=0)
fig=plt.figure()
plt.plot(distances)
plt.xlim(99000, 100300)

In [ ]:
# test
dbscan_cluster_model = DBSCAN(eps=0.009, min_samples=605, algorithm='ball_tree', metric='haversine').fit(np.radians(X))
dbscan_cluster_model
dbscan_cluster_model.labels_
US_picked_df['cluster'] = dbscan_cluster_model.labels_

In [ ]:
# score of test
ss(X, US_picked_df['cluster'])
# below is testing range of min_s and eps

epsilons = np.linspace(0.009,0.011, num=3)
min_samples = np.arange(100, 700, step=55) 
combinations = list(itertools.product(epsilons, min_samples))
combinations
N = len(combinations)
# find best model
def get_scores_and_labels(combinations, X):
  scores = []
  all_labels_list = []
  

  for i, (eps, num_samples) in enumerate(combinations):
    
    dbscan_cluster_model = DBSCAN(eps= eps, min_samples= num_samples, algorithm='ball_tree', metric='haversine').fit(np.radians(X))
    labels = dbscan_cluster_model.labels_
    labels_set = set(labels)
    num_clusters = len(labels_set)
    if -1 in labels_set:
      num_clusters -= 1
    
    if (num_clusters < 2) or (num_clusters > 50):
      scores.append(-10)
      all_labels_list.append('bad')
      c = (eps, num_samples)
      print(f"Combination {c} on iteration {i+1} of {N} has {num_clusters} clusters. Moving on")
      continue
    
    scores.append(ss(X, labels))
    all_labels_list.append(labels)
    print(f"Index: {i}, Score: {scores[-1]}, Labels: {all_labels_list[-1]}, NumClusters: {num_clusters}")

  best_index = np.argmax(scores)
  best_parameters = combinations[best_index]
  best_labels = all_labels_list[best_index]
  best_score = scores[best_index]

  return {'best_epsilon': best_parameters[0],
          'best_min_samples': best_parameters[1], 
          'best_labels': best_labels,
          'best_score': best_score}

best_dict = get_scores_and_labels(combinations, X)
US_picked_df['cluster'] = best_dict['best_labels']
best_dict

In [ ]:
# find centroid for each cluster
involved = US_picked_df['cluster'] != -1
US_involved = US_picked_df[involved]
US_involved


Xx = US_involved[['latitude','longitude']].to_numpy()
lablel = US_involved[['cluster']]
num_clusters = len(set(best_dict['best_labels']) - set([-1]))
cluster_labels = best_dict['best_labels']
clusters = pd.Series([X[cluster_labels == n] for n in range(num_clusters)])
clusters

def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

# get the centroid point for each cluster
centermost_points = clusters.map(get_centermost_point)
lats, lons = zip(*centermost_points)
rep_points = pd.DataFrame({'lon':lons, 'lat':lats})
rep_points

In [ ]:
#visualisation 
location = US_picked_df['latitude'].mean(), US_picked_df['longitude'].mean()

m = folium.Map(location=location,zoom_start=4,control_scale = True)

folium.TileLayer('cartodbpositron').add_to(m)

clust_colours = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99','#b15928']

for i in range(0,len(US_picked_df)):
    colouridx = US_picked_df['cluster'].iloc[i]
    if colouridx != -1:
         folium.CircleMarker([US_picked_df['latitude'].iloc[i],US_picked_df['longitude'].iloc[i]], radius = 4, color = "white", fill = "white").add_to(m)
         
         col = clust_colours[colouridx%len(clust_colours)]
         folium.CircleMarker([US_picked_df['latitude'].iloc[i],US_picked_df['longitude'].iloc[i]], radius = 4, color = col, fill = col).add_to(m)
        
    else:
       
        folium.CircleMarker([US_picked_df['latitude'].iloc[i],US_picked_df['longitude'].iloc[i]], radius = 4, color = "white", fill = "white").add_to(m)
for i in range(len(rep_points)):
    folium.CircleMarker([rep_points['lat'].iloc[i],rep_points['lon'].iloc[i]], radius = 3, color = "black", fill = "black").add_to(m)      
        
m
m.save("US.html")

In [ ]:
#davies_bouldin_score and calinski_harabasz_score
db_index = davies_bouldin_score(X, best_dict['best_labels'])
print(db_index)
ch_index = calinski_harabasz_score(X, best_dict['best_labels'])
print(ch_index)

---------------------------------------------------------------------------

# Text Similarity

### Filter out short accident descriptions and common words
Only descriptions over 130 characters are kept, and the NLTK stopwords are used to filter out common words from the descriptions

In [ ]:
stop = stopwords.words('english')

df_longer_info = df[df['Description'].str.len()>130]

# ensure all descriptions are of type string
df_longer_info['Description'] = df_longer_info['Description'].astype('string')
# remove stop words
df_longer_info['Description'] = df_longer_info['Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [ ]:
# Get number of accident, and pairs of accidents for written report
n = len(df_longer_info)
print("# of accident reports: ", n)
print("# of pairs of accident reports: ", math.comb(n,2))

### Define shingling and hashing functions

In [ ]:
# makes shingles of length 8
def shingles(word, n = 8):
    return [word[i:i + n] for i in range(len(word) - n + 1)]

#hashes a list of strings (from class notes)
def listHash(l,seed):
	val = 0
	for e in l:
		val = min(val, mmh3.hash(e, seed))
	return val 

### Shingle the description texts and create a zipped list of accident ID and shingles

In [ ]:
df_longer_info['shingles'] = df_longer_info.Description.map(shingles)

description_index = list(zip(df_longer_info['ID'], df_longer_info['shingles']))

### Define signatures and Jaccard functions

In [ ]:
def signatures(zipped_dict, seeds):

    sig_dict = {k: [] for k in df_longer_info['ID']}

    for entry in zipped_dict:
        sig_dict[entry[0]] = []

        for seed in seeds:
            # create a dictionary with accident IDs as index for later reference
            sig_dict[entry[0]].append(listHash(entry[1], seed))
    
    return sig_dict

def jaccard(list1, list2):

    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    
    return float(intersection) / union

### Create unique seeds for hashing the signatures 
A signature dictionary as well as an ordered list are saved for future use

In [ ]:
seeds = set()

while len(seeds) < 100:
    seeds.add(np.random.randint(0,1000))

signature_dict = signatures(description_index, seeds)
signature_dict_list = list(signature_dict.items())

### Create an LSH function 
This function divides the signatures into b bands and then saves the banded signatures to a dictionary list

In [ ]:
def lsh(signature_dict_list, b):
    n = len(signature_dict_list[0][1])
    # ensure that the number of signatures can be evenly divided by the number of bands
    assert n % b == 0
    r = int(n/b)
    band_dict = {} 
    for i in signature_dict_list:
        band_dict[i[0]] = []
        for j in range(0, n, r):
            #change to append hash
            band_dict[i[0]].append(i[1][j:j+r])
    return list(band_dict.items())

### Apply the LSH and search for texts that have bands hashed to the same bucket
The loop below will stop comparing two items as soon as one band has been found in the same bucket, these then become a candidate pair (*the code below can take ~5 minutes to run*)

In [ ]:
# number of bands is set to 10, because 10 bands of 10 rows gives a ~0.8 threshold
band_dict = lsh(signature_dict_list, 10)

result = []

for item in list(itertools.combinations(band_dict, 2)):
    for sig_band1, sig_band2 in zip(item[0][1], item[1][1]):
        if sig_band1==sig_band2:
            data = [item[0][0], item[1][0], signature_dict[item[0][0]], signature_dict[item[1][0]]]
            result.append(data)
            continue

df1 = pd.DataFrame(result, columns=['ID1', 'ID2', 'Sigs_1', 'Sigs_2'])

df1['similarity'] = df1.apply(lambda x: jaccard(x.Sigs_1, x.Sigs_2), axis=1)

# print dataframe with pairs that have over 0.8 similarity but less than 1.0
print(df1[(df1['similarity']> 0.8) & (df1['similarity']!=1.0)])

---------------------------------------------------------------------------

# Neural Network

In [ ]:
#load data
df = pd.read_csv('US_accidents_May19_Migrated Data.csv')

In [ ]:
#Get a look at the data we're working with
df

We make a histogram of the severity column, and see that the values 2,3,4 are massively overrepresented as compared to values 0 and 1. 1 is still somewhat represented, but there are almost no datapoints with severity 0.

In [ ]:
plt.hist(df['Severity'],bins=5)
plt.yscale('log')

In [ ]:
#See what features we have in the datset
df.columns.values

Time should be an important feature for predicting the severity of an accident. One would expect that more serious accidents happen at night and during the winter for example. We can encode the time into a few different features: the year of the accident, the month, the hour of the day, and the weekday. Additionally, from the start_time and End_Time features, we can calculate how long the accident stop the flow of traffic, as a long traffic stoppage likely means that the accident was more severe.

In [ ]:
#duration feature using start and end time
df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')
df['End_Time'] = pd.to_datetime(df['End_Time'], errors='coerce')

# Extract year, month, day, hour and weekday
df['Year']=df['Start_Time'].dt.year
df['Month']=df['Start_Time'].dt.month

df['Hour']=df['Start_Time'].dt.hour
df['Weekday']=df['Start_Time'].dt.strftime('%a')
# Extract the amount of time in the unit of minutes for each accident, round to the nearest integer
df['Duration']=round((df['End_Time']-df['Start_Time'])/np.timedelta64(1,'m'))

#adapted from https://medium.com/@vaibhavgope02/predicting-accident-severity-with-us-accidents-dataset-4aeaaae0b0af

In [ ]:
set(df['Weekday'].values)

Select which features we would like to keep:

In [ ]:
Good_columns = ['Year','Month','Hour','Weekday','Duration', 'count of Bump','Count of Crossing','count Traffic Signal','Crossing','Junction','No_Exit','Railway','Roundabout','Side','Station','Stop','Sunrise_Sunset','Traffic_Calming','Traffic_Signal','Temperature(F)','Visibility(mi)','Weather_Condition','Humidity(%)','Precipitation(in)','Pressure(in)','Wind_Speed(mph)','Start_Lat','Start_Lng','Severity']
#List of all used columns:
#the features listed below contains strings or true/false values that needs to be transformed:
to_be_transformed = ['Crossing','Junction','No_Exit','Railway','Roundabout','Side','Station','Stop','Sunrise_Sunset','Traffic_Calming','Traffic_Signal','Weather_Condition','Weekday']

In [ ]:
new_df=df[Good_columns]
#fill 0-values in the Precipitation feature, since we assume that the lack of data means that there was no rainfall.
new_df['Precipitation(in)']=new_df['Precipitation(in)'].fillna(value=0)
new_df = new_df.dropna(axis=0,how='any').reset_index()


Weather conditions has many different values, so we one hot encode it, since one-hot encoding usually works well for learning categorical features with deep learning models since the model can intepret higher numbers in ordinal features as more important.

In [ ]:
#One-hot encoding for the Weather_Condition parameter
y = pd.get_dummies(new_df.Weather_Condition, prefix='Weather_Condition')
y

In [ ]:
new_df = pd.concat([new_df,y],axis=1)
new_df = new_df.drop('Weather_Condition',axis=1)
new_df.columns

We could also want to one-hot encode Years, however the result in the report was achieved without one-hot encoded Years.

In [ ]:
#One hot encode the Year feature we made earlier
y = pd.get_dummies(new_df.Year, prefix='Year')
y
new_df = pd.concat([new_df,y],axis=1)
new_df = new_df.drop('Year',axis=1)
new_df.columns

We can also one-hot encode "Weekday". This was done rather than the cyclical transformation presented later for the model shown in the report.

In [ ]:
#One hot encode the Weekday feature we made earlier
y = pd.get_dummies(new_df.Weekday, prefix='Weekday')
y
new_df = pd.concat([new_df,y],axis=1)
new_df = new_df.drop('Weekday',axis=1)
new_df.columns

In [ ]:
#Check for NaN-values:
new_df.isnull().values.any()

In [ ]:
#change "true/false" features to binary, ordinal features, and categorical features to ordinal features.

#
to_be_transformed = []
for column in tqdm(new_df.columns):
    if type(new_df[column][0])==type('string'):
        to_be_transformed.append(column)
    elif new_df[column][0]==True or new_df[column][0]==False: #replace "true" with 1 and "false" with 0
        new_df[column]=new_df[column].astype(int)
#print which features still need to be transformed
print(to_be_transformed)
transform_dic = {}
for column in tqdm(to_be_transformed): #loop over the remaining columns
    dic = {}
    count = 0
    for item in new_df[column]: #loop over each element in the column
        if item == True:
            dic[item]=1
        elif item == False:
            dic[item]=0
        elif item not in dic:
            dic[item]=count
            count+=1
    transform_dic[column]=dic
#Now that we have the new values for the transformed columns we assign the new values
for column in tqdm(to_be_transformed):
    new_list = []
    for i in range(len(new_df[column])):
        new_list.append(transform_dic[column][new_df[column][i]])
    new_df[column]=new_list

In [ ]:
backup_df=new_df.copy()#IGNORE THIS

In [ ]:
#Since there are almost 0 cases where the severity is 0, we remove the points that have the value. This will likely improve performance.
new_df.drop(new_df[new_df.Severity==0].index,inplace=True)


In [ ]:
#sanity check, check that transformation worked
set(new_df['Weekday'].values)

For the time features, we can use cyclical encoding. Cyclical encoding maps the features to sine and cosine. It is useful for time features, since hour 24 is essentially the same as hour 0. Based on the following article:https://www.kaggle.com/code/avanwyk/encoding-cyclical-features-for-deep-learning
This was not done in the run shown in the report.

In [ ]:
new_df['Hour_sin'] = np.sin(2*np.pi*new_df['Hour']/24.0)
new_df['Hour_cos'] = np.cos(2*np.pi*new_df['Hour']/24.0)
new_df['Month_sin'] = np.sin(2*np.pi*new_df['Month']/12)
new_df['Month_cos'] = np.cos(2*np.pi*new_df['Month']/12)
new_df['Weekday_sin'] = np.cos(2*np.pi*new_df['Weekday']/7)
new_df['Weekday_cos'] = np.cos(2*np.pi*new_df['Weekday']/7)


In [ ]:
new_df.drop(['index','Hour','Month','Weekday'],axis=1,inplace=True)#drop old time columns

In [ ]:
#save finished dataframe as pickle. If running training code on google colab, upload this to google drive.
with open('final_df.pickle', 'wb') as handle:
    pickle.dump(new_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Feed Forward Neural Network

In [ ]:
from google.colab import drive #For google colab integration
drive.mount('/content/drive')

First we load the data:


In [ ]:
!pip3 install pickle5
import pickle5 as pickle
# df = pd.read_csv('/content/drive/MyDrive/Computational Tools/final_df234.csv')
with open('/content/drive/MyDrive/Computational_Tools/final_df234.pickle', 'rb') as f:
  df = pickle.load(f)

In [ ]:
n = 1500000 #Sample size
df_subset = df.sample(n).copy()
#split into data and targets:
X = df_subset.drop(columns = ['Severity']).copy() 
y = df_subset['Severity']

In [ ]:
#Normalize data
normalized_df=(X-X.mean())/X.std()

#Split data into training(95%), validation(2.5%) and test split(2.5%):
X_train, X_rem, y_train, y_rem = train_test_split(X,y,train_size = 0.95)

X_test, X_val, y_test, y_val = train_test_split(X_rem,y_rem, test_size=0.5)

#the data set contains far more datapoints of severity 2 then the other values, making severity 2 the most common category. 
#To avoid this messing with the outcome, we try undersampling classes Severity 2 and 3, and oversampling category 1.
train_data = pd.concat([X_train, y_train], axis=1)

#Separate the dataset by class
Sev1 = train_data[train_data.Severity==1]
Sev2 = train_data[train_data.Severity==2]
Sev3 = train_data[train_data.Severity==3]
Sev4 = train_data[train_data.Severity==4]
Majority_list = [Sev1,Sev2,Sev3]
#Set over and undersampling ratios
# over_under_size = [int(a*(len(Sev4)/50000)) for a in [10000,100000,70000]]
over_under_size = [len(Sev4),len(Sev4),len(Sev4)]
downsampled = Sev4
for i in range(len(Majority_list)):
  Maj_downsampled = resample(Majority_list[i], replace=True,  n_samples=over_under_size[i], random_state=27) #resample data
  downsampled = pd.concat([downsampled,Maj_downsampled],axis=0)
#Set data to tensors of correct datatype:
X_train = torch.Tensor(downsampled.drop(columns = ["Severity"]).copy().values).type(torch.float64)
y_train = torch.Tensor(downsampled['Severity'].values).type(torch.LongTensor)-1 #We subtract one so the smallest class is 0.
X_test = torch.Tensor(X_test.values).type(torch.float64)
X_val = torch.Tensor(X_val.values).type(torch.float64)
y_test = torch.Tensor(y_test.values)-1
y_val = torch.Tensor(y_val.values)-1


In [ ]:
#Check the size of the training and validation set:
print(len(X_train),len(y_val))

We can check if the data can predict the class label with a simple linear regression. We do this to get an idea of how difficult the problem is to solve.

In [ ]:
#We use an sklearn implementation of linear regression since we only want to check if the problem is easily solvable:
model = lm.LinearRegression()
model.fit(X_train,y_train)

# Predict alcohol content
y_est = model.predict(X_test)
residual = y_est-y_test.numpy()

# Display scatter plot
figure()
subplot(2,1,1)
plt.plot(y_test, y_est, '.')
plt.xlabel('Severity (true)'); plt.ylabel('Severity (estimated)');
  
subplot(2,1,2)
plt.hist(residual,bins=[0,0.5,1.5,2.5,3.5,4])

show()

The problem is not easily solved, since the linear regression cannot distinguish between the classes

Below we build the network(adapted from 02456 exercise 3.3 2021)

In [ ]:
#Hyperparameters
num_output = 4
num_l1 = 1024
num_l2 = 2048
num_features = X_train.shape[1]

# define network
class Net(nn.Module):
  def __init__(self,num_features, num_hidden, num_hidden_2, num_output):
    super(Net, self).__init__()
    self.linear1 = nn.Linear(num_features,num_l1)
    self.linear2 = nn.Linear(num_l1,num_l2)
    self.linear3 = nn.Linear(num_l2,num_l1)
    self.linear4 = nn.Linear(num_l1,num_output)
    # self.linear3 = nn.Linear(num_l2,num_output)


    # self.activation = torch.nn.Tanh()
    self.activation = torch.nn.ReLU()
    self.dropout = torch.nn.Dropout(p=0.6)
    self.batchnorm1 = torch.nn.BatchNorm1d(num_hidden)
    self.batchnorm2 = torch.nn.BatchNorm1d(num_hidden_2)

  def forward(self, x):
    x = self.linear1(x)
    x = self.dropout(x)
    x = self.activation(x)
    x = self.batchnorm1(x)
    x = self.linear2(x)
    x = self.dropout(x)
    x = self.activation(x)
    x = self.batchnorm2(x)
    x = self.linear3(x)
    x = self.dropout(x)
    x = self.batchnorm1(x)
    x = self.linear4(x)
    return F.softmax(x,dim=1)

net = Net(num_features, num_l1, num_l2, num_output).double()

if torch.cuda.is_available():
    print('##converting network to cuda-enabled')
    net.cuda()
print(net)

In [ ]:
#Define the optimizer and criterion(Loss function)
optimizer = optim.Adam(net.parameters(), lr=1e-4, weight_decay = 1e-5)
criterion = nn.CrossEntropyLoss()


Below we have the training loop for the network (adapted from 02456 exercise 3.3)
In order to run this code, cuda will have to be available on your machine. Otherwise it can run on google colab using a GPU runtime.

In [ ]:


# setting hyperparameters and gettings epoch sizes
batch_size = 4096
num_epochs = 100
num_samples_train = X_train.shape[0]
num_batches_train = num_samples_train // batch_size
num_samples_valid = X_val.shape[0]
num_batches_valid = num_samples_valid // batch_size

# setting up lists for handling loss/accuracy
train_acc, train_loss = [], []
valid_acc, valid_loss = [], []
test_acc, test_loss = [], []
cur_loss = 0
losses = []

get_slice = lambda i, size: range(i * size, (i + 1) * size)

for epoch in range(num_epochs):
    # Forward -> Backprob -> Update params
    ## Train
    cur_loss = 0
    net.train()
    for i in range(num_batches_train):


        slce = get_slice(i, batch_size)
        input = X_train[slce].cuda()
        # input = input.cuda()
        output = net(input)
        
        # compute gradients given loss
        target_batch = y_train[slce]
        target_batch = target_batch.cuda()
        batch_loss = criterion(output, target_batch)
        # batch_loss.requires_grad=True
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        
        cur_loss += batch_loss  
    losses.append(cur_loss / batch_size)

    net.eval()
    ### Evaluate training
    train_preds, train_targs = [], []
    for i in range(num_batches_train):
        slce = get_slice(i, batch_size)
        input = X_train[slce].cuda()
        output = net(input)
        
        preds = torch.max(output, 1)[1]
        
        train_targs += list(y_train[slce].cpu().numpy())
        train_preds += list(preds.data.cpu().numpy())
    
    ### Evaluate validation
    val_preds, val_targs = [], []
    for i in range(num_batches_valid):
        slce = get_slice(i, batch_size)
        input = X_val[slce].cuda()
        output = net(input)
        preds = torch.max(output, 1)[1]
        val_targs += list(y_val[slce].cpu().numpy())
        val_preds += list(preds.data.cpu().numpy())
        

    train_acc_cur = accuracy_score(train_targs, train_preds)
    valid_acc_cur = accuracy_score(val_targs, val_preds)
    
    train_acc.append(train_acc_cur)
    valid_acc.append(valid_acc_cur)
    
    if epoch % 10 == 0:
        print("Epoch %2i : Train Loss %f , Train acc %f, Valid acc %f" % (
                epoch+1, losses[-1], train_acc_cur, valid_acc_cur))

epoch = np.arange(len(train_acc))
plt.figure()
plt.plot(epoch, train_acc, 'r', epoch, valid_acc, 'b')
plt.legend(['Train Accucary','Validation Accuracy'])
plt.xlabel('Updates'), plt.ylabel('Acc')

Please not that due to the randomness involved in sampling data points, splitting the dataset and initializing parameters, results of training may not be exactly the same as reported in the paper.

Below we plot the loss curve for the training:

In [ ]:
losses = [loss.cpu().detach().numpy() for loss in losses]
plt.figure()
plt.plot(epoch, losses)
plt.legend(['Loss'])
plt.xlabel('Updates') #, plt.ylabel('')


Then we can plot a confusion matrix to tell how our model performed, and in what mistakes the network made, if any.

In [ ]:
net.cpu()
preds = torch.max(net(X_test), 1)[1]
cm = confusion_matrix(y_test,preds)
df_cm = pd.DataFrame(cm, index = range(1,5), columns = range(1,5))
plt.figure(figsize=(10,7))
sn.heatmap(df_cm, annot=True)
plt.title('Test Accuracy: {}'.format(accuracy_score(y_test,preds)))